In [8]:
#Used to find subsets of lists
import itertools

In [9]:
##Helper functions
#Updates endogenous variables (V) given a certain state X,W
def V_given_set(V,X_or_W,indices):
    updated_V = V.copy()
    var_seen = 0
    for i in range(len(V)):
        if i in indices:
            updated_V[i] = X_or_W[var_seen]
            var_seen += 1
    return updated_V
    
#Creates a list of all subsets of a given lst
def subsets_finder(lst,indices):
    subsets_of_lst = []
    for i in range(1,(len(indices))):
        subsets_of_lst.append(list(itertools.combinations(indices, i)))
    return subsets_of_lst
    
#Given certain indices, sets lst[i] = False for i in indices
def update_var(lst,indices):
    return_lst = lst.copy()
    for i in range(len(lst)):
        if i in indices:
            return_lst[i] == False
    return return_lst


#Created a class of original H.P defn of Causality
class Causal_Model:
    #U,V,F are assumed to be lists for simplicity sake
    #U,V are boolean values corresp. to some list of var
    #F is a list of pointers to functions (e.g. def foo())
    def __init__(self, U, V, R, F):
        self.exogenous_var = U
        self.endogenous_var = V
        self.var_range = R
        self.function = F
        self.signature = (U,V,R)
        self.model = (self.signature,F)
    
    # For the following, I assume that this can be used to check causality if an outcome did not occur(?)
    
    #AC1 of defn, checks if outcome = function for given X
    #outcome_val refers to desired outcome
    def ac1_check(self,X,X_indices,outcome_function,outcome_val):
        V = self.endogenous_var
        outcome_given_X = outcome_function(self.exogenous_var,V_given_set(V,X,X_indices))
        return outcome_given_X == outcome_val
    
    #AC2(a) checks the but-for clause, i.e changing X would lead to opposite outcome
    def ac2_a_check(self,X,X_indices,outcome_function,outcome_val):
        x_prime = [not i for i in X]
        V_given_x_prime = V_given_set(V,x_prime,X_indices)  
        return outcome_val != outcome_function(self.exogenous_var,V_given_x_prime)
    
    
    #Checks AC2(b) of the defn
    #Checks that outcome holds for all other W, fixing Z
    def ac2_b_check(self,Z,Z_indices,W, W_indices,outcome_function, outcome_val):
        V_fixed_Z = V_given_set(V,Z,Z_indices)
        subsets_of_W = subsets_finder(W,W_indices)
        for i in subsets_of_W:
            curr_V = V_given_set(V_fixed_Z,W,i)
            updated_W = update_var(W,indices)
            outcome = outcome_function(V,self.exogenous_var,curr_V)
            if outcome_val != outcome:
                return False
        return True
    
    #Checks that X is minimal by iterating over all subsets
    def ac3_check(self,X,X_indices,outcome_function,outcome_val):
        subsets_of_X = subsets_finder(X,X_indices)
        for i in subsets_of_X:
            updated_X = update_var(X,X_indices)
            curr_V = V_given_set(self.endogenous_var,X,i)
            if outcome_val == outcome_function(self.exogenous_var,curr_V):
                return False
        return True
    
    #Returns true if X satisfies HP defn, False o.w.
    def causality_check(self,Z,Z_indices,X,X_indices,W,W_indices,outcome_val,outcome_func):
        ac_1 = self.ac1_check(X,X_indices,outcome_func,outcome_val)
        ac2_a = self.ac2_a_check(X,X_indices,outcome_func,outcome_val)
        ac2_b= self.ac2_b_check(Z,Z_indices,W, W_indices,outcome_func, outcome_val)
        ac_3 = self.ac3_check(X,X_indices,outcome_func,outcome_val)
        return ac_1 and ac2_a and ac2_b and ac_3

## Testing
### Using examples from 2005 HP paper

### Example 2.1: Forest Fire (F) caused by either Lightning (L) or Match Lit (ML) 

In [10]:
#Initializes U to True for random exogen. var (val/var not important)
#Initializes V to False (can also do True)
#R is trivially set to 0 or 1
# Index[0] = L ; Index[1] = ML ; Index[2] = F
U = [True,True,True]
V = [False,False,False]
R = [(0,1),(0,1),(0,1)]
def forest_fire(U,V):
    return V[0] or V[1]
F_f = forest_fire
F = [None, None, F_f]
FF_model = Causal_Model(U,V,R,F)

In [11]:
Z = [True]
Z_index = [0]
W = [False]
W_index = [1]
print("Testing Casuality of Lightning (Correct Val = True):")
print(FF_model.causality_check(Z,Z_index,Z,Z_index,W,W_index,True,F_f))

Testing Casuality of Lightning (Correct Val = True):
True


In [12]:
Z_ml = [True]
Z_ml_index = [1]
W_ml = [False]
W_index = [0]
print("Testing for ML (Correct Val = True):")
print(FF_model.causality_check(Z_ml,Z_ml_index,Z_ml,Z_ml_index,W_ml,W_index,True,F_f))

Testing for ML (Correct Val = True):
True


In [13]:
Z_both = [True,True]
Z_both_index = [0,1]
W_both = []
W_both_index = []
print("Testing for Both (Correct Val = True):")
print(FF_model.causality_check(Z_both,Z_both_index,Z_both,Z_both_index,W_both,W_both_index,True,F_f))

Testing for Both (Correct Val = True):
True
